## 1. Primeros pasos con Spark

Vamos a utilizar la librería PySpark, que es la API de Python para interactuar con la shell de Spark. Se puede instalar la librería mediante la siguiente guía:

- https://www.sicara.ai/blog/2017-05-02-get-started-pyspark-jupyter-notebook-3-minutes

In [8]:
# Importar librerías y dependencias
import findspark
findspark.init()

import pyspark
from pyspark import SparkContext, SparkConf

### 1.1. SparkContext

El punto de entrada de cualquier programa PySpark es un objeto SparkContext, este objeto le indica como conectarse a un clúster de Spark y crear RDDs. Para crer un SparkContext, primero es necesario cosntruir un objeto SparkConf conteniendo la información de la aplicación.

```Python
conf = SparkConf().setAppName('myApp').setMaster('local[*]')
sc = SparkContext(conf=conf)
```

El parámetro `local[*]` es una cadena especial que indica que está utilizando un clúster local. El asterisco * le indica a Spark que cree tantos hilos de trabajo como cores tenga la máquina.

In [5]:
conf = SparkConf().setAppName('myApp').setMaster('local[*]')
sc = SparkContext(conf=conf)

21/11/18 19:21:09 WARN Utils: Your hostname, gmachin resolves to a loopback address: 127.0.1.1; using 192.168.1.43 instead (on interface enp4s0)
21/11/18 19:21:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/18 19:21:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# sc.setLogLevel('WARN')

Con este objeto, podemos acceder a la SparkUI:

In [6]:
sc

<SparkContext master=local[*] appName=myApp>

Y consultar el resto de configuraciones por defecto:

In [7]:
sc.getConf().getAll()

[('spark.driver.port', '36985'),
 ('spark.app.name', 'myApp'),
 ('spark.app.id', 'local-1637259671044'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.startTime', '1637259669891'),
 ('spark.driver.host', '192.168.1.43'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

### 1.2. RDD

Spark gira en torno al concepto de *Resilient Distributed Dataset* (RDD), que es una colección de elementos que se pueden operar en paralelo. Hay dos formas de crear un RDD: paralelizar una colección existente, o hacer referencia a un conjunto de datos en un sistema de almacenamiento externo, como un sistema de archivos compartido, HDFS, HBase o cualquier fuente de datos que ofrezca un formato de entrada Hadoop.

Para crear un RDD se utiliza el método `parallelize` del objeto SparkContext.

In [13]:
# Crear un RDD
rdd = sc.parallelize(range(10000000))
rdd

PythonRDD[1] at RDD at PythonRDD.scala:53

Un parámetro importante para los RDD es el número de particiones para distribuir el conjunto de datos. Spark ejecutará una task para cada partición del clúster. Por lo general, se utilizan de 2 a 4 particiones para cada CPU del clúster. Normalmente, Spark infiere la cantidad de particiones automáticamente en función de los recursos. Sin embargo, también puede configurarlo manualmente pasándolo como un segundo parámetro de parallelize.

In [7]:
# Ver las particiones de un RDD (valor 4 por defecto)
rdd.getNumPartitions()

4

In [10]:
# Reparticionar un RDD
rdd_repartitioned = rdd.repartition(8)
rdd_repartitioned.getNumPartitions()

8

### 1.3. Operaciones con RDD

Los RDDs soportan dos tipos de operaciones:

- transformaciones
- acciones

Las transformaciones tienen como consecuencia la creación de un nuevo dataset a partir de uno ya existente, mientras que las acciones, devuelven un valor al driver del cluster después de realizar alguna operación sobre el dataset. La función `map()` por ejemplo sería una transformación, ya que aplica una función a cada valor de un dataset; por otro lado, la función `reduce()` devuelve un único valor fruto de algún tipo de agregación.

Las transformaciones en Spark se denominan *lazy*, ya que no se ejecutan hasta que se aplica una acción sobre el dataset. Esta característica permite que Spark se ejecute de una forma más eficiente. Por ejemplo si sobre un dataset realizamos un `map()` y luego un `reduce()`, al driver únicamente se le informará del valor de la acción y no de todos los pasos anteriores.

In [16]:
# Ejemplo de operaciones con un RDD
# ------------------------------------------------------------------------------
# Definimos un puntero al fichero
lines = sc.textFile("ficheros/extracto-quijote.txt")

# Definimos un nuevo RDD sobre el puntero anterior
lineLengths = lines.map(lambda s: len(s))

lineLengths

PythonRDD[6] at RDD at PythonRDD.scala:53

Si intentamos ver el contenido de lineLengths, veremos que no aparece nada, y solo da información del tipo de objeto que es, aún no se ha realizado ninguna operación.

In [18]:
# Realizamos una acción sobre el RDD
totalLength = lineLengths.reduce(lambda a, b: a + b)

totalLength

176

Finalmente, realizamos una acción sobre el RDD lineLengths. Es en este punto, donde Spark comienza a ejecutar todo el trabajo sobre el RDD, para finalmente enviar un valor al driver del cluster. Si por un casual se fuera a trabajar con el RDD lineLengths de forma repetida, se puede almacenar en memoria mediante el método `persist()`.

In [19]:
lineLengths.persist()

PythonRDD[6] at RDD at PythonRDD.scala:53

## 2. Bibliografía

- https://spark.apache.org/docs/latest/rdd-programming-guide.html